## PROYETO PILOTO AUTONOMA DESERCION

Luego de revisar varias veces la data, se decide empezar por la tabla con mayor detalle que es EstudianteAsignatura, que tiene información no solo de las asignaturas que el estudiante ha matriculado en un periodo, sino también qué grupos ha tomado de dicha asigantura e incluso qué docentes dictaron clase en cada grupo.

La tabla base del proyecto es EstudianteSemestre, que es la que indica si el estudiante tomó asignaturas en ese semestre o no.   Llegaremos posteriormente a ella.

Usaremos solo una conexión a la base de datos.  Solo ejecutar el siguiente codigo una vez.

Para queries solo se requiere lo siguiente y el resultado queda en el dataframe
1. query = 'SELECT ...'
2. df = pd.read_sql(query, cnx)

Para DML se requiere:
1. crear un cursor = cnx.cursor()
2. query = 'update...'
3. cursor.execute(query)
4. cnx.commit()
5. cursor.close()

In [18]:
from sqlalchemy import create_engine
import pandas as pd

usuario = 'uaopiloto'
contraseña = 'ap'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'UAOPiloto'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contraseña}@{host}/{base_de_datos}'
conexionMySQL = create_engine(cadena_conexion)


# df.to_sql(nombre_de_tabla, motor, if_exists='replace', index=False)

# import mysql.connector
# import pandas as pd

# # Conectarse a la base de datos 
# cnx = mysql.connector.connect(user='uaopiloto', password='ap',
#                               host='localhost',
#                               database='UAOPiloto')

ALGORITMO e HIPOTESIS

1. creamos PERIODOS a partir de los distintos períodos en AXM y que nos queden ordenados, y en una lista
pues luego vamos a confirmar si luego de dos períodos estos estudiantes han dejado de matricular.
1. creamos DATOS a partir de Matrícula.
2. adicionamos la columna LABEL que debe ser 1 si luego de 2 periodos,  el estudiante dejó de matricular mínimo 2 periodos consecutivos o más y no fue que se graduó.  De lo contrario debe ser 0.

BASE:  puede ser que nos paramos al final de un semestre, o al prinpcio de un semestre.    Este segundo caso nos permitiria jugar con 202302.  Asi que escogemos asumir que estamos iniciando un semestre y por tanto no tenemos notas del semestre actual.

HIPOTESIS

1.  promedio acumulado de 4 semestres anteriores
2.  promedio por semestre de 4 semestres anteriores
3.  cantidad de asignaturas que tomó el semestre actual y 4 semestres hacia atras
4.  cantidad de créditos que tomó semestre actual y 4 semestres anteriores
5.  cantidad de créditos que perdió 4 semestres anteriores
6.  cantidad de asignaturas que perdió 4 semestres anteriores
7.  si asistió a actividades extracurriculares 4 semestres anteriores
8.  que tan heterogenoes fueron los grupos que toma este semestre y 4 anteriores (desvStd edad, relación F:M)
9.  tuvo mora en 4 semestres anteriores
10. docentes - edad promedio docentes este semestre y anteriores
11. asignaturas que tomó y sus notas en 4 semestres anteriores (son un jurgo de variables)


Algoritmo Paso 1:  creamos PERIODOS

In [34]:
# Traemos los PERIODOS
query = """
SELECT distinct PERIODO 
FROM EstudianteAsignatura
ORDER BY PERIODO"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
PERIODOS = pd.read_sql_query(query, conexionMySQL)
lPeriodos = PERIODOS['PERIODO'].to_list()

# Treamos una lista de estudiantes
query = """
SELECT distinct id_estudiante 
FROM EstudianteAsignatura
ORDER BY id_estudiante"""

# Leer resultados en un DataFrame y lo convertimos en una lsita
ESTUDIANTES = pd.read_sql_query(query, conexionMySQL)
lEstudiantes = ESTUDIANTES['id_estudiante'].to_list()



Algoritmo Paso 2:  cremos DATOS a partir EstudianteAsignatura(id_estudiante, periodo)

In [71]:
# Hacer una consulta SQL
query = """
SELECT distinct id_estudiante, periodo 
FROM EstudianteAsignatura"""

# Leer resultados en un DataFrame
MATRICULAS = pd.read_sql_query(query, conexionMySQL)
MATRICULAS.shape

(832, 2)

Algoritmo paso 3:  crear columna LABEL y que sea 1 si dado una flia en matricula, NO hay fila en matrícula durante 2 periodos consecutivos posteriores.

In [72]:
MATRICULAS['LABEL'] = 0

for e in range(0, len(lEstudiantes)):

    # print('Estudiante:', lEstudiantes[e])

    for p in range(0, len(lPeriodos)-2):  # los ultimos 2 períodos no los analizamos

        if ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p])).any():

            # si ocurre un error es porque los indices se salieron de madre
            try:
                semestre_1 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+1])).any() 
                semestre_2 = ((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p+2])).any() 
            except IndexError:
               semestre_1 = False
               semestre_2 = False

            if semestre_1 & semestre_2:    
                # print(      'si matriculó ambos periodos')           
                pass 
            else:
                # print(      'no matriculó ambos períodos')
                MATRICULAS.loc[((MATRICULAS['id_estudiante'] == lEstudiantes[e]) & (MATRICULAS['periodo'] == lPeriodos[p])),['LABEL']] = 1
                break

Hay que revisar de los registros que quedaron marcados como desertores (LABEL==1), que no ocurra que fue que el estudiante se graduó.

TAREA
Tomamos esos registros, les adicionamos la fecha de grado si la tienen, convertida a formato YYYYNN, donde NN = 01 o 03 si se graduó en el primer o segundo semestre
y miramos que el PERIODO P que aparece como desertor no ven a ser que en p+1 o p+2 se graduó...  





In [73]:
import numpy as np
# hacer outerjoin entre MATRICULAS y GRADUADOS para meter la 
# fecha de grado
# luego hacer un for con esos registros que tienen fecha de grado no nula
# y convertirla al formato YYYYNN donde NN=01 o 03 según el grado haya ocurrido
# en alguno de esos semestres.

# si periodo_grado < periodo +2 quiere decir que se graduó ANTES de haber 
# pasado dos semestres y por tanto no puede mardarse como desertor.

query = """
SELECT id_estudiante, fecha_grado 
FROM estudiante
WHERE fecha_grado is not null"""

GRADUADOS = pd.read_sql_query(query, conexionMySQL)

# creamos periodo_grado en el semestre en que este ocurrió (01 o 03)
GRADUADOS['periodo_grado'] = np.where(GRADUADOS['fecha_grado'].dt.month <= 6, 
                   GRADUADOS['fecha_grado'].dt.year.astype(str) + '01',
                   GRADUADOS['fecha_grado'].dt.year.astype(str) + '03')

MATRICULAS = pd.merge(MATRICULAS, GRADUADOS, on=['id_estudiante'], how='left')
MATRICULAS[MATRICULAS['periodo_grado'].notnull()]

# hacer un for sobre MATRICULAS.  Detectar un caso con 1 en LABEL.   
# mirar si existe como graduado en el semestre p + 1 o p + 2.
# si ocurre, LABEL = 0

                   

,id_estudiante,periodo,LABEL,fecha_grado,periodo_grado
137,2185298,201803,0,2022-11-04,202203
138,2185298,202001,0,2022-11-04,202203
139,2185298,202201,0,2022-11-04,202203
140,2185298,202103,1,2022-11-04,202203
141,2185298,202003,0,2022-11-04,202203
142,2185298,202101,0,2022-11-04,202203
143,2185298,201901,0,2022-11-04,202203
144,2185298,201903,0,2022-11-04,202203
621,2185221,201901,0,2023-05-06,202301
622,2185221,201803,0,2023-05-06,202301
